# Dataset

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df = pd.read_csv("https://raw.githubusercontent.com/eduardofc/data/main/amazon_sports.csv")
df['review_body'] = df['review_body'].str.replace("[^a-zA-Zñáéíóú .,]", "", regex=True)
df['review_body'] = df['review_body'].str.lower()
df.head()

,stars,review_body,review_title,product_category
0,1,nunca llego el pedido y el vendedor pasa de todo no contestan,No llego nunca,sports
1,1,"no sé como es, porque debería haber llegado ayer día de marzo, y hoy por la noche sigo esperando que llegue el frontal. y me habéis mandado el formulario de opinión. pues de momento mala no ha cumplido con la fecha de entrega.",Todavía no ha llegado,sports
2,1,"guantes cómodos, no lo niego, pero de mala calidad. yo creo que en caso de caída no valdrian para mucho, dos meses de uso y se están rajando.",Guantes de baja calidad,sports
3,1,hasta hoy no he visto el producto. el pedido hace ya casi mes. y notifico que he usado prime para está compra.,Muy Mala experiencia,sports
4,1,"no puedo valorarla porque, después de casi una semana, aún no he recibido mi pedido. pienso que amazon tendría que valorar las compañías de transporte con que trabaja, porque es indignante que pague mi cuota prime y nunca reciba mi pedido el día que toca",Paquete perdido?,sports


In [2]:
df = df[df.stars!=3]
df['good_product'] = (df.stars>3).astype(int)
df.groupby('good_product').size()

good_product
0    4989
1    5372
dtype: int64

# Text Classification

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [4]:
X = df.review_body.values
y = df.good_product

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=33)

print(len(X_train))
print(len(X_test))

8288
2073


In [5]:
# tokenizer

vocab_size = 5000

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

In [6]:
tokenized_train = tokenizer.texts_to_sequences(X_train)
tokenized_test = tokenizer.texts_to_sequences(X_test)

In [7]:
# padding

max_length = 50

padded_train = pad_sequences(tokenized_train, maxlen=max_length)
padded_test = pad_sequences(tokenized_test, maxlen=max_length)

In [8]:
padded_train

array([[   8,    1,    3, ...,   47, 2120,  270],
       [   0,    0,    0, ...,   16,   35, 1514],
       [   0,    0,    0, ...,  831,   31,   32],
       ...,
       [   0,    0,    0, ...,   43,  138,  270],
       [   0,    0,    0, ...,  623,   39,   53],
       [   0,    0,    0, ...,  687,    3, 1765]])

# Models

In [9]:
import tensorflow.keras as keras
import numpy as np
from time import time

from keras import Sequential
from keras.layers import *
from keras.optimizers import Adam

In [13]:
def compile_model(model):
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics='accuracy'
    )
#     model.summary()

def fit_model(model):
    model.fit(
        padded_train, y_train,
        epochs=20,
        batch_size=100,
        validation_data=(padded_test, y_test)
    )

## Model 1: MLP

In [14]:
keras.utils.set_random_seed(812)

model = Sequential([
    Flatten(input_shape=(max_length,)),
    Dense(128, activation='relu'),
    Dense(54, activation='relu'),
    Dense(1, activation='sigmoid')
])
compile_model(model)

In [15]:
fit_model(model)

Epoch 1/20
83/83 [==============================] - 1s 4ms/step - loss: 47.3593 - accuracy: 0.5048 - val_loss: 23.0460 - val_accuracy: 0.4954
Epoch 2/20
83/83 [==============================] - 0s 3ms/step - loss: 18.2673 - accuracy: 0.5332 - val_loss: 15.9072 - val_accuracy: 0.5065
Epoch 3/20
83/83 [==============================] - 0s 2ms/step - loss: 11.8044 - accuracy: 0.5556 - val_loss: 13.4263 - val_accuracy: 0.5080
Epoch 4/20
83/83 [==============================] - 0s 2ms/step - loss: 8.3238 - accuracy: 0.5893 - val_loss: 11.9636 - val_accuracy: 0.5051
Epoch 5/20
83/83 [==============================] - 0s 2ms/step - loss: 6.1279 - accuracy: 0.6053 - val_loss: 11.1099 - val_accuracy: 0.5128
Epoch 6/20
83/83 [==============================] - 0s 2ms/step - loss: 5.0438 - accuracy: 0.6248 - val_loss: 9.9673 - val_accuracy: 0.5205
Epoch 7/20
83/83 [==============================] - 0s 2ms/step - loss: 4.4862 - accuracy: 0.6357 - val_loss: 9.5762 - val_accuracy: 0.5099
Epoch 8/20
8

## Model 2: MLP + Embeddings

In [18]:
keras.utils.set_random_seed(812)

embd_size=20

model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embd_size,
        input_length=max_length
    ), 
    Flatten(),
    Dense(1, activation='sigmoid')
])
compile_model(model)

In [19]:
fit_model(model)

Epoch 1/20
83/83 [==============================] - 1s 7ms/step - loss: 0.6788 - accuracy: 0.5685 - val_loss: 0.6575 - val_accuracy: 0.6454
Epoch 2/20
83/83 [==============================] - 0s 3ms/step - loss: 0.5844 - accuracy: 0.7396 - val_loss: 0.5197 - val_accuracy: 0.7752
Epoch 3/20
83/83 [==============================] - 0s 3ms/step - loss: 0.4301 - accuracy: 0.8343 - val_loss: 0.4164 - val_accuracy: 0.8215
Epoch 4/20
83/83 [==============================] - 0s 3ms/step - loss: 0.3378 - accuracy: 0.8733 - val_loss: 0.3757 - val_accuracy: 0.8350
Epoch 5/20
83/83 [==============================] - 0s 3ms/step - loss: 0.2810 - accuracy: 0.8977 - val_loss: 0.3557 - val_accuracy: 0.8495
Epoch 6/20
83/83 [==============================] - 0s 3ms/step - loss: 0.2412 - accuracy: 0.9167 - val_loss: 0.3461 - val_accuracy: 0.8529
Epoch 7/20
83/83 [==============================] - 0s 3ms/step - loss: 0.2069 - accuracy: 0.9319 - val_loss: 0.3428 - val_accuracy: 0.8514
Epoch 8/20
83/83 [==

## Model 3: CNN + Embeddings

In [27]:
keras.utils.set_random_seed(812)

embd_size = 20
ks = 5
n_filters = 64

model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embd_size,
        input_length=max_length
    ), 
    Conv1D(kernel_size=ks, filters=n_filters),
    GlobalAveragePooling1D(),
    Dense(6, activation='relu'),
    Dense(1, activation='sigmoid')
])
compile_model(model)

In [28]:
fit_model(model)

Epoch 1/20
83/83 [==============================] - 1s 7ms/step - loss: 0.6594 - accuracy: 0.5938 - val_loss: 0.5665 - val_accuracy: 0.7795
Epoch 2/20
83/83 [==============================] - 0s 5ms/step - loss: 0.4138 - accuracy: 0.8341 - val_loss: 0.3795 - val_accuracy: 0.8394
Epoch 3/20
83/83 [==============================] - 0s 5ms/step - loss: 0.2833 - accuracy: 0.8891 - val_loss: 0.3511 - val_accuracy: 0.8534
Epoch 4/20
83/83 [==============================] - 0s 5ms/step - loss: 0.2242 - accuracy: 0.9183 - val_loss: 0.3651 - val_accuracy: 0.8534
Epoch 5/20
83/83 [==============================] - 0s 5ms/step - loss: 0.1891 - accuracy: 0.9336 - val_loss: 0.3944 - val_accuracy: 0.8534
Epoch 6/20
83/83 [==============================] - 0s 5ms/step - loss: 0.1586 - accuracy: 0.9472 - val_loss: 0.4233 - val_accuracy: 0.8495
Epoch 7/20
83/83 [==============================] - 0s 5ms/step - loss: 0.1357 - accuracy: 0.9579 - val_loss: 0.4690 - val_accuracy: 0.8413
Epoch 8/20
83/83 [==

## Model 4: (Bi)-LSTM + Embeddings

In [32]:
keras.utils.set_random_seed(812)

embd_size = 20
lstm_dim = 32

model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embd_size,
        input_length=max_length
    ), 
#     LSTM(lstm_dim),
    Bidirectional(LSTM(lstm_dim)),
    Dense(1, activation='sigmoid')
])
compile_model(model)

In [33]:
fit_model(model)

Epoch 1/20
83/83 [==============================] - 6s 31ms/step - loss: 0.6330 - accuracy: 0.6248 - val_loss: 0.4954 - val_accuracy: 0.8119
Epoch 2/20
83/83 [==============================] - 2s 21ms/step - loss: 0.3883 - accuracy: 0.8415 - val_loss: 0.3702 - val_accuracy: 0.8442
Epoch 3/20
83/83 [==============================] - 2s 21ms/step - loss: 0.2664 - accuracy: 0.8959 - val_loss: 0.3548 - val_accuracy: 0.8476
Epoch 4/20
83/83 [==============================] - 2s 21ms/step - loss: 0.2076 - accuracy: 0.9235 - val_loss: 0.3786 - val_accuracy: 0.8529
Epoch 5/20
83/83 [==============================] - 2s 21ms/step - loss: 0.1784 - accuracy: 0.9361 - val_loss: 0.4224 - val_accuracy: 0.8490
Epoch 6/20
83/83 [==============================] - 2s 21ms/step - loss: 0.1395 - accuracy: 0.9531 - val_loss: 0.4434 - val_accuracy: 0.8452
Epoch 7/20
83/83 [==============================] - 2s 21ms/step - loss: 0.1191 - accuracy: 0.9616 - val_loss: 0.5092 - val_accuracy: 0.8302
Epoch 8/20
83

## Model 5: quitar overfitting

In [35]:
keras.utils.set_random_seed(812)

embd_size = 20
lstm_dim = 32

model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embd_size,
        input_length=max_length
    ), 
    Dropout(.5),
    LSTM(lstm_dim),    
    Dropout(.5),
    BatchNormalization(),
    Dense(6, activation='relu'),
    Dense(1, activation='sigmoid')
])
compile_model(model)

In [37]:
fit_model(model)

Epoch 1/20
83/83 [==============================] - 1s 18ms/step - loss: 0.0932 - accuracy: 0.9669 - val_loss: 0.8431 - val_accuracy: 0.7824
Epoch 2/20
83/83 [==============================] - 1s 17ms/step - loss: 0.0889 - accuracy: 0.9680 - val_loss: 0.6598 - val_accuracy: 0.8312
Epoch 3/20
83/83 [==============================] - 1s 18ms/step - loss: 0.0842 - accuracy: 0.9662 - val_loss: 0.6627 - val_accuracy: 0.8321
Epoch 4/20
83/83 [==============================] - 1s 17ms/step - loss: 0.0781 - accuracy: 0.9716 - val_loss: 0.7025 - val_accuracy: 0.8215
Epoch 5/20
83/83 [==============================] - 1s 18ms/step - loss: 0.0750 - accuracy: 0.9725 - val_loss: 0.7679 - val_accuracy: 0.8341
Epoch 6/20
83/83 [==============================] - 1s 18ms/step - loss: 0.0714 - accuracy: 0.9726 - val_loss: 0.8239 - val_accuracy: 0.8022
Epoch 7/20
83/83 [==============================] - 1s 17ms/step - loss: 0.0701 - accuracy: 0.9753 - val_loss: 0.7417 - val_accuracy: 0.8225
Epoch 8/20
83

# Tamaño del dataset

In [38]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df1 = pd.read_csv("https://raw.githubusercontent.com/eduardofc/data/main/amazon_sports.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/eduardofc/data/main/amazon_home.csv")
df3 = pd.read_csv("https://raw.githubusercontent.com/eduardofc/data/main/amazon_electronics.csv")

df = pd.concat([df1, df2, df3])

df['review_body'] = df['review_body'].str.replace("[^a-zA-Zñáéíóú .,]", "", regex=True)
df['review_body'] = df['review_body'].str.lower()
df.head()

,stars,review_body,review_title,product_category
0,1,nunca llego el pedido y el vendedor pasa de todo no contestan,No llego nunca,sports
1,1,"no sé como es, porque debería haber llegado ayer día de marzo, y hoy por la noche sigo esperando que llegue el frontal. y me habéis mandado el formulario de opinión. pues de momento mala no ha cumplido con la fecha de entrega.",Todavía no ha llegado,sports
2,1,"guantes cómodos, no lo niego, pero de mala calidad. yo creo que en caso de caída no valdrian para mucho, dos meses de uso y se están rajando.",Guantes de baja calidad,sports
3,1,hasta hoy no he visto el producto. el pedido hace ya casi mes. y notifico que he usado prime para está compra.,Muy Mala experiencia,sports
4,1,"no puedo valorarla porque, después de casi una semana, aún no he recibido mi pedido. pienso que amazon tendría que valorar las compañías de transporte con que trabaja, porque es indignante que pague mi cuota prime y nunca reciba mi pedido el día que toca",Paquete perdido?,sports


In [39]:
df = df[df.stars!=3]
df['good_product'] = (df.stars>3).astype(int)
df.groupby('good_product').size()

good_product
0    20512
1    19817
dtype: int64

In [40]:
X = df.review_body.values
y = df.good_product

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=33)

print(len(X_train))
print(len(X_test))

32263
8066


In [41]:
# tokenizer

vocab_size = 5000

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

In [42]:
tokenized_train = tokenizer.texts_to_sequences(X_train)
tokenized_test = tokenizer.texts_to_sequences(X_test)

In [43]:
# padding

max_length = 50

padded_train = pad_sequences(tokenized_train, maxlen=max_length)
padded_test = pad_sequences(tokenized_test, maxlen=max_length)

In [44]:
keras.utils.set_random_seed(812)

embd_size = 20
lstm_dim = 32

model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embd_size,
        input_length=max_length
    ), 
    Dropout(.5),
    LSTM(lstm_dim),    
    Dropout(.5),
    BatchNormalization(),
    Dense(6, activation='relu'),
    Dense(1, activation='sigmoid')
])
compile_model(model)

In [45]:
fit_model(model)

Epoch 1/20
323/323 [==============================] - 8s 18ms/step - loss: 0.4539 - accuracy: 0.7791 - val_loss: 0.4749 - val_accuracy: 0.7966
Epoch 2/20
323/323 [==============================] - 5s 17ms/step - loss: 0.3195 - accuracy: 0.8665 - val_loss: 0.3177 - val_accuracy: 0.8662
Epoch 3/20
323/323 [==============================] - 5s 17ms/step - loss: 0.2882 - accuracy: 0.8803 - val_loss: 0.2993 - val_accuracy: 0.8758
Epoch 4/20
323/323 [==============================] - 6s 17ms/step - loss: 0.2673 - accuracy: 0.8878 - val_loss: 0.2862 - val_accuracy: 0.8797
Epoch 5/20
323/323 [==============================] - 5s 17ms/step - loss: 0.2527 - accuracy: 0.8959 - val_loss: 0.3120 - val_accuracy: 0.8690
Epoch 6/20
323/323 [==============================] - 5s 17ms/step - loss: 0.2401 - accuracy: 0.9008 - val_loss: 0.2887 - val_accuracy: 0.8866
Epoch 7/20
323/323 [==============================] - 5s 17ms/step - loss: 0.2312 - accuracy: 0.9052 - val_loss: 0.2743 - val_accuracy: 0.8856